In [1]:
import tensorflow as tf
import numpy as np
import random

# как обычно, фиксируем сид
np.random.seed(17)
tf.random.set_seed(17)
random.seed(17)

In [3]:
densnet = tf.keras.applications.DenseNet201()

82526208/82524592 [==============================] - 7s 0us/step


In [4]:
densnet.summary()

Model: "densenet201"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [6]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [13]:
cat_img = tf.keras.preprocessing.image.load_img('CNN/cat.jpg', target_size=(224, 224))

In [33]:
cat_img.show()

AttributeError: 'numpy.ndarray' object has no attribute 'show'

In [14]:
cat_img = tf.keras.preprocessing.image.img_to_array(cat_img)
cat_img = np.expand_dims(cat_img, axis=0)
cat_img = tf.keras.applications.densenet.preprocess_input(cat_img)

In [16]:
pred = densnet.predict(cat_img)

In [17]:
tf.keras.applications.densenet.decode_predictions(pred)

40960/35363 [==================================] - 0s 1us/step


[[('n02124075', 'Egyptian_cat', 0.16923514),
  ('n01882714', 'koala', 0.1480275),
  ('n01883070', 'wombat', 0.11369833),
  ('n02108915', 'French_bulldog', 0.10923684),
  ('n02125311', 'cougar', 0.04562815)]]

In [19]:
for layer in densnet.layers:
    layer.trainable = False

# но несколько последних разморозим обратно
# число 20 здесь взято для примера, а вообще, конечно,
# это тоже гиперпараметр, который нужно подбирать
# оптимальным может оказаться любое значение от 1 слоя до переобучения половины сети

for layer in densnet.layers[-10:]:
    layer.trainable = True

# заменим активацию на последнем слое

#resnet.layers[-1].activation = tf.keras.activations.relu

In [20]:
model_cats = tf.keras.models.Sequential([
    densnet,  # вся модель выступает в качестве слоя
    tf.keras.layers.Dense(1, activation='sigmoid')  # слой для бинарной классификации
])

In [21]:
model_cats.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 1000)              20242984  
_________________________________________________________________
dense (Dense)                (None, 1)                 1001      
Total params: 20,243,985
Trainable params: 2,204,625
Non-trainable params: 18,039,360
_________________________________________________________________


In [22]:
accuracy = tf.keras.metrics.binary_accuracy
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

# как и в прошлый раз, F1 напишем сами
def f1_metrics(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * ((prec * rec) / (prec + rec + 1e-7))


model_cats.compile(optimizer=tf.keras.optimizers.Adam(),
                   loss=tf.keras.losses.binary_crossentropy,
                   metrics=[accuracy, precision, recall, f1_metrics])

In [23]:
import os

In [24]:
def preprocess_image(file):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(224, 224))  # загружаем в нужном разрешении
    img = tf.keras.preprocessing.image.img_to_array(img)  # конвертируем в массив
    img = tf.keras.applications.resnet.preprocess_input(img)  # препроцессинг для resnet
    return img

In [25]:
# добавляем пары (картинка, 1) для картинок с котами
cats = [(preprocess_image('CNN/pics/cats/'+file), 1) for file in os.listdir('CNN/pics/cats')]

# и пары (картинка, 0) для картинок без котов
nocats = [(preprocess_image('CNN/pics/nocats/'+file), 0) for file in os.listdir('CNN/pics/nocats')]

In [26]:
all_pics = cats + nocats
random.shuffle(all_pics)

In [27]:
x = np.array([a[0] for a in all_pics])
y = np.array([a[1] for a in all_pics])

In [28]:
def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1 - val_frac - test_frac) * len(x))]
    x_val = x[round((1 - val_frac - test_frac) * len(x)):round((1 - test_frac) * len(x))]
    x_test = x[round((1 - test_frac) * len(x)):]
    return x_train, x_val, x_test


x_train, x_val, x_test = train_val_test_split(x)
y_train, y_val, y_test = train_val_test_split(y)

In [29]:
# будем отслеживать обучение в Tensorboard

tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/tl_resnet_cats', histogram_freq=1)

# и уменьшать lr на плато

annealing = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)

In [32]:
bs = 16  # размер батча

# вместо самих данных подаём в цикл обучения картинки из нашего генератора
model_cats.fit(x_train, y_train,
               validation_data=(x_val, y_val),
               steps_per_epoch=len(x_train)/bs,  # чтобы генератор не уходил в бесконечный цикл, указываем количество шагов
               epochs=50,
               callbacks=[tb_callback, annealing])

Epoch 1/50
20/20 [==============================] - 79s 3s/step - loss: 0.6881 - binary_accuracy: 0.5726 - precision: 0.3366 - recall: 0.1359 - f1_metrics: 0.2567 - val_loss: 0.6762 - val_binary_accuracy: 0.6912 - val_precision: 1.0000 - val_recall: 0.2500 - val_f1_metrics: 0.3918
Epoch 2/50
20/20 [==============================] - 58s 3s/step - loss: 0.6713 - binary_accuracy: 0.7354 - precision: 0.9164 - recall: 0.2196 - f1_metrics: 0.3494 - val_loss: 0.6619 - val_binary_accuracy: 0.8382 - val_precision: 0.9474 - val_recall: 0.6429 - val_f1_metrics: 0.7907
Epoch 3/50
20/20 [==============================] - 60s 3s/step - loss: 0.6667 - binary_accuracy: 0.7144 - precision: 0.8655 - recall: 0.2353 - f1_metrics: 0.4195 - val_loss: 0.6608 - val_binary_accuracy: 0.7941 - val_precision: 0.8889 - val_recall: 0.5714 - val_f1_metrics: 0.6965
Epoch 4/50
20/20 [==============================] - 60s 3s/step - loss: 0.6592 - binary_accuracy: 0.7672 - precision: 0.8726 - recall: 0.4501 - f1_metrics